In [ ]:
import pandas as pd
import numpy as np
from pandas_datareader import data
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import math
import six
import sys
sys.modules['sklearn.externals.six'] = six

import mlrose

import yfinance as yf
yf.pdr_override()

dataset = pd.read_csv('db/acoes1.csv')
dataset


In [ ]:
dinheiro_total = 5000
taxa_selic_2010 = 12.5
taxa_selic_2011 = 9.25
taxa_selic_2012 = 7.25
taxa_selic_2013 = 7.75
taxa_selic_2014 = 9.5
taxa_selic_2015 = 14.25
taxa_selic_2016 = 14.25
taxa_selic_2017 = 14.25
taxa_selic_2018 = 6.5
taxa_selic_2019 = 5.25
taxa_selic_2020 = 2.25
taxa_selic_2021 = 7.75
taxa_selic_2022 = 11.75
taxa_selic_2023 = 12.5

valor_2010 = dinheiro_total + (dinheiro_total * taxa_selic_2010 / 100)
valor_2011 = valor_2010 + (dinheiro_total * taxa_selic_2011 / 100)
valor_2012 = valor_2011 + (dinheiro_total * taxa_selic_2012 / 100)
valor_2013 = valor_2012 + (dinheiro_total * taxa_selic_2013 / 100)
valor_2014 = valor_2013 + (dinheiro_total * taxa_selic_2014 / 100)
valor_2015 = valor_2014 + (dinheiro_total * taxa_selic_2015 / 100)
valor_2016 = valor_2015 + (dinheiro_total * taxa_selic_2016 / 100)
valor_2017 = valor_2016 + (dinheiro_total * taxa_selic_2017 / 100)
valor_2018 = valor_2017 + (dinheiro_total * taxa_selic_2018 / 100)
valor_2019 = valor_2018 + (dinheiro_total * taxa_selic_2019 / 100)
valor_2020 = valor_2019 + (dinheiro_total * taxa_selic_2020 / 100)
valor_2021 = valor_2020 + (dinheiro_total * taxa_selic_2021 / 100)
valor_2022 = valor_2021 + (dinheiro_total * taxa_selic_2022 / 100)
valor_2023 = valor_2022 + (dinheiro_total * taxa_selic_2023 / 100)
valor_2023
taxa_selic_historico = np.array([taxa_selic_2010, taxa_selic_2011, taxa_selic_2012, taxa_selic_2013, taxa_selic_2014, taxa_selic_2015, taxa_selic_2016, taxa_selic_2017, taxa_selic_2018, taxa_selic_2019, taxa_selic_2020, taxa_selic_2021, taxa_selic_2022, taxa_selic_2023])
dataset_original = dataset.copy()
dinheiro_total = 5000
sem_risco = taxa_selic_historico.mean() / 100


# (dataset['taxa de retorno'].mean() - taxa_selic_historico.mean()/100)/ dataset['taxa de retorno'].std() * math.sqrt(246) #sharp ratio validando historico da selic


In [ ]:
def fitness_function(solucao):
  dataset = dataset_original.copy()
  pesos = solucao / solucao.sum()

  for i in dataset.columns[1:]:
    dataset[i] = (dataset[i] / dataset[i][0])

  for i, acao in enumerate(dataset.columns[1:]):
    dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

  dataset.drop(labels = ['Date'], axis = 1, inplace=True)
  dataset['soma valor'] = dataset.sum(axis = 1)
  dataset['taxa retorno'] = 0.0

  for i in range(1, len(dataset)):
    dataset['taxa retorno'][i] = ((dataset['soma valor'][i] / dataset['soma valor'][i - 1]) - 1) * 100

  sharpe_ratio = ((dataset['taxa retorno'].mean() - sem_risco) / dataset['taxa retorno'].std()) * np.sqrt(246)

  return sharpe_ratio

In [ ]:
def alocacao_ativos(dataset, dinheiro_total, seed = 0, melhores_pesos = []):
    if seed != 0:
        np.random.seed(seed)

    dataset = dataset.copy()

    if len(melhores_pesos) > 0:
        pesos = melhores_pesos
    else:  
        pesos = np.random.random(len(dataset.columns) - 1)
    #print(pesos, pesos.sum())
        pesos = pesos / pesos.sum()
    #print(pesos, pesos.sum())
    
 
    colunas = dataset.columns[1:]

    for i in colunas:
        dataset[i] = dataset[i] / dataset[i][0]

    for i, acao in enumerate(dataset.columns[1:]):
        dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

    dataset['SOMA VALOR'] = dataset[colunas].sum(axis=1)

    datas = dataset['Date']

    dataset.drop(columns=['Date'], inplace=True)
    dataset['taxa de retorno'] = 0.0

    for i in range(1, len(dataset)):
        dataset['taxa de retorno'][i] = ((dataset['SOMA VALOR'][i] / dataset['SOMA VALOR'][i-1]) - 1 )*100

    acoes_pesos = pd.DataFrame(data= {'Ações': colunas, 'Pesos': pesos*100})

    return dataset, datas, acoes_pesos , dataset.loc[len(dataset)-1]['SOMA VALOR']

In [ ]:
np.random.seed(10)
pesos = np.random.random(len(dataset_original.columns) - 1)
pesos = pesos / pesos.sum()
pesos

In [ ]:
def visualiza_alocacao(solucao):
  colunas = dataset_original.columns[1:]
  for i in range(len(solucao)):
    print(colunas[i], solucao[i] * 100)
visualiza_alocacao(pesos)    

In [ ]:
fitness = mlrose.CustomFitness(fitness_function)

In [ ]:
problema_maximizacao = mlrose.ContinuousOpt(length=23, fitness_fn=fitness,
                                            maximize = True, min_val = 0.1, max_val = 1)

In [ ]:
problema_minimizacao = mlrose.ContinuousOpt(length=23, fitness_fn=fitness,
                                            maximize = False, min_val = 0, max_val = 1)

In [ ]:
melhor_solucao, melhor_custo = mlrose.hill_climb(problema_maximizacao, random_state = 1) ## hill climb
melhor_solucao, melhor_custo

In [ ]:
melhor_solucao = melhor_solucao / melhor_solucao.sum()
melhor_solucao, melhor_solucao.sum()

In [ ]:
visualiza_alocacao(melhor_solucao)

In [ ]:
_, _, _, soma_valor = alocacao_ativos(pd.read_csv('db/acoes1.csv'), 5000, melhores_pesos=melhor_solucao)
soma_valor

In [ ]:
## Simulated annealing
melhor_solucao, melhor_custo = mlrose.simulated_annealing(problema_maximizacao, random_state = 1) 
melhor_solucao = melhor_solucao / melhor_solucao.sum()
melhor_solucao, melhor_custo

In [ ]:
visualiza_alocacao(melhor_solucao)

In [ ]:
_, _, _, soma_valor = alocacao_ativos(pd.read_csv('db/acoes1.csv'), 5000, melhores_pesos=melhor_solucao)
soma_valor

In [37]:
#algoritmo genético 
problema_maximizacao_ag = mlrose.ContinuousOpt(length = 23, fitness_fn = fitness, 
                                               maximize = True, min_val = 0, max_val = 1)
melhor_solucao, melhor_custo = mlrose.genetic_alg(problema_maximizacao_ag, random_state = 1, pop_size=200, mutation_prob= 0.05, max_attempts = 10)
melhor_solucao = melhor_solucao / melhor_solucao.sum()
melhor_solucao, melhor_custo


In [ ]:
visualiza_alocacao(melhor_solucao)

In [ ]:
_, _, _, soma_valor = alocacao_ativos(pd.read_csv('db/acoes1.csv'), 5000, melhores_pesos=melhor_solucao)
soma_valor